Load data

In [6]:
import json


file_path = '12-11-game-states.json'

game_states = []

with open(file_path, 'r') as file:
    for line in file:
        game_states.append(json.loads(line.strip()))

print(f"Loaded {len(game_states)} game states.")

Loaded 12480 game states.


In [ ]:
def extract_pokemon_data(pokemon_data):
    return {
        "name": pokemon_data["mon"],
        "hp_percent": pokemon_data["hp_percent"],
        "status": pokemon_data["status"],
        "terratype": pokemon_data["terratype"]
    }
#fucntion to extract data from the game state
def process_game_state(game_state):
    player = game_state["player"]
    opponent = game_state["opponent"]
    

    active_player_pokemon = extract_pokemon_data(player["active_pokemon"])
    active_opponent_pokemon = extract_pokemon_data(opponent["active_pokemon"])
    

    player_seen = [extract_pokemon_data(pokemon) for pokemon in player["seen_pokemon"].values()]
    opponent_seen = [extract_pokemon_data(pokemon) for pokemon in opponent["seen_pokemon"].values()]

    player_effects = player["player_side_effects"]
    opponent_effects = opponent["opponent_side_effects"]
    player_boosts = player["boosts"]
    opponent_boosts = opponent["boosts"]
    
    

    battle_state = game_state["battle_state"]
    

    est_winrate = game_state["est_winrate"]
    
    return {
        "active_player_pokemon": active_player_pokemon,
        "active_opponent_pokemon": active_opponent_pokemon,
        "player_seen": player_seen,
        "opponent_seen": opponent_seen,
        "player_effects": player_effects,
        "opponent_effects": opponent_effects,
        "player_boosts": player_boosts,
        "opponent_boosts": opponent_boosts,
        "battle_state": battle_state,
        "est_winrate": est_winrate
    }

In [ ]:
processed_data = [process_game_state(gs) for gs in game_states]
import numpy as np
import pandas as pd

df = pd.DataFrame(processed_data)
df.to_csv('processed_game_states.csv', index=False)

Embedding the pokemon names along side their data

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

#+2 for dead and dontknow
name_vocab_size = 509  # Update based on unique Pokémon names I think there are 507

embedding_dim = 32      # Tune based on dataset size


def create_pokemon_input(mon_num):
    name_input = Input(shape=(1,), name=f"{mon_num}_name")  
    hp_input = Input(shape=(1,), name=f"{mon_num}_hp")  #
    status_input = Input(shape=(6,), name=f"{mon_num}_status")  
    terra_input = Input(shape=(18,), name=f"{mon_num}_terra")  

    name_embedding = layers.Embedding(name_vocab_size, embedding_dim, name=f"{mon_num}_name_embedding")(name_input)
    name_embedding = layers.Flatten()(name_embedding)


    combined = layers.Concatenate()([name_embedding, hp_input, status_input, terra_input])
    return name_input, hp_input, status_input, terra_input, combined

player_inputs = []
opponent_inputs = []
player_combined = []
opponent_combined = []

for i in range(6):  
    pname, php, pstatus, pterra, pcomb = create_pokemon_input(f"player_poke{i+1}")
    oname, ohp, ostatus, oterra, ocomb = create_pokemon_input(f"opponent_poke{i+1}")

    player_inputs.extend([pname, php, pstatus, pterra])
    opponent_inputs.extend([oname, ohp, ostatus, oterra])
    player_combined.append(pcomb)
    opponent_combined.append(ocomb)




model rough draft idea

In [ ]:
# create a function to get each one and concatenate
combined_features = layers.Concatenate()([player_features, opponent_features, side_effects, boosts, battle_state]) 


x = layers.Dense(128, activation="relu")(combined_features)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.3)(x)

output = layers.Dense(1, activation="sigmoid", name="win_rate")(x)

model = Model(inputs=player_inputs + opponent_inputs + [side_effects, boosts, battle_state], outputs=output)


model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [16]:
y = df['est_winrate']

X = df.drop(columns=['est_winrate']) 

from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)